In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, when, sum, count
from pyspark.sql.types import IntegerType, DoubleType, StringType




In [ ]:
#Inicar sesión
spark = SparkSession.builder.appName("CreditRisk").getOrCreate()

In [ ]:
#Definición de rutas
r_transactions= "FileStore/raw/credit_transactions.csv"
r_accounts = "FileStore/raw/raw/customer_accounts.csv"
r_demographics = "FileStore/raw/raw/customer_demographics.csv"
r_merchant = "FileStore/raw/merchant_categories.csv"


In [ ]:
#Función de lectura de archivo csv
def upload_csv(ruta):
    return spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("delimiter", ",").load(ruta)

In [ ]:
#BRONZE LAYER

#Validaciones Iniciales: Verificar IDs nulos
df_transactions = upload_csv(r_transactions).filter(col("transaction_id").isNotNull())
df_accounts = upload_csv(r_accounts).filter(col("customer_id").isNotNull())
df_demographics = upload_csv(r_demographics).filter(col("customer_id").isNotNull())
df_merch_categories = upload_csv(r_merchant).filter(col("category").isNotNull())


In [ ]:
#Guardar tablas en formato optimizado
df_transactions.write.mode("overwrite").parquet("FileStore/bronze/credit_transactions")
df_accounts.write.mode("overwrite").parquet("FileStore/bronze/customer_accounts")
df_demographics.write.mode("overwrite").parquet("FileStore/bronze/customer_demographics")
df_merch_categories.write.mode("overwrite").parquet("FileStore/bronze/merchant_categories")